In [48]:
from predict_utils import tokenizer, openStopword, openModel, vectorize
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import csv
import pickle
import joblib

samsung_csv = pd.read_csv('dataset/Samsung_noQuestion.csv', encoding="utf-8")
hyundai_csv = pd.read_csv('dataset/Hyundai_noQuestion.csv', encoding="utf-8")
CJ_csv = pd.read_csv('dataset/CJ_noQuestion.csv', encoding="utf-8")
SK_csv = pd.read_csv('dataset/SK_noQuestion.csv', encoding="utf-8")
LG_csv = pd.read_csv('dataset/LG_noQuestion.csv', encoding="utf-8")


In [49]:

for i in range (0, len(samsung_csv)):
    samsung_csv["document"][i] = samsung_csv["document"][i].replace("\n\r","").replace("\r","")
    
for i in range (0, len(hyundai_csv)):
    hyundai_csv["document"][i] = hyundai_csv["document"][i].replace("\n\r","").replace("\r","")
    
for i in range (0, len(CJ_csv)):
    CJ_csv["document"][i] = CJ_csv["document"][i].replace("\n\r","").replace("\r","")
    
for i in range (0, len(SK_csv)):
    SK_csv["document"][i] = SK_csv["document"][i].replace("\n\r","").replace("\r","")
    

for i in range (0, len(LG_csv)):
    LG_csv["document"][i] = LG_csv["document"][i].replace("\n\r","").replace("\r","")




In [50]:
samsung_csv["label"] = "samsung"
hyundai_csv["label"] = "hyundai"
CJ_csv["label"] = "CJ"
SK_csv["label"] = "SK"
LG_csv["label"] = "LG"

In [51]:
okt = Okt()

all_data = samsung_csv.append(hyundai_csv).append(CJ_csv).append(SK_csv).append(LG_csv)


In [52]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(all_data, test_size=0.33, random_state=42)

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [44]:
#SVM 모델

text_clf_svm = Pipeline([
    ('vect', vectorize),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=1000, random_state=42))]) # 손실함수 기댓값 최소화

text_clf_svm = text_clf_svm.fit(train["document"].apply(lambda x: np.str_(x)), train["label"])

joblib.dump(text_clf_svm, 'Company_SVM_2.joblib')

['Company_SVM_2.joblib']

In [45]:
train = np.mean(text_clf_svm.predict(train["document"].apply(lambda x: np.str_(x))) == train["label"])
test = np.mean(text_clf_svm.predict(test["document"].apply(lambda x: np.str_(x))) == test["label"])

In [46]:
with open('Company_SVM_2.txt', 'w') as f:
    f.write(str(train))
    f.write(str(test))
    f.close()

In [ ]:
# SVC (ver. prob, gamma='scale')
SVC_PROB_pipeline = Pipeline([
                ('tfidf', vectorize),
                ('clf', OneVsRestClassifier(SVC(probability=True, gamma='scale'), n_jobs=1))])

text_clf_svc_prob = SVC_PROB_pipeline.fit(train["document"].apply(lambda x: np.str_(x)), train["label"])


joblib.dump(text_clf_svc_prob, 'Company_SVC_scale.j_2oblib')

In [ ]:
train = np.mean(text_clf_svc_prob.predict(train["document"].apply(lambda x: np.str_(x))) == train["label"])
test = np.mean(text_clf_svc_prob.predict(test["document"].apply(lambda x: np.str_(x))) == test["label"])

In [ ]:
with open('Company_SVC_scale_2.txt', 'w') as f:
    f.write(str(train))
    f.write(str(test))
    f.close()

In [ ]:

# SVC (ver. prob, gamma='auto')
SVC_PROB_pipeline = Pipeline([
                ('tfidf', vectorize),
                ('clf', OneVsRestClassifier(SVC(probability=True, gamma='auto'), n_jobs=1)), # n_jobs = -1 로 하면 오류남.
            ])
text_clf_svc_prob = SVC_PROB_pipeline.fit(train["document"].apply(lambda x: np.str_(x)), train["label"])

joblib.dump(text_clf_svc_prob, 'Company_SVC_auto_2.joblib')

##### train = np.mean(text_clf_svc_prob.predict(train["document"].apply(lambda x: np.str_(x))) == train["label"])
test = np.mean(text_clf_svc_prob.predict(test["document"].apply(lambda x: np.str_(x))) == test["label"])

In [ ]:
with open('Company_SVC_auto_2.txt', 'w') as f:
    f.write(str(train))
    f.write(str(test))
    f.close()

In [6]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
# CalibratedClassifierCV, cv=10
SVC_pipeline_Calibrated = Pipeline([
                ('tfidf',vectorize),
                ('clf', CalibratedClassifierCV(LinearSVC(), cv=10))
            ])
clf_svc_calibrated = SVC_pipeline_Calibrated.fit(train["document"].apply(lambda x: np.str_(x)), train["label"])

joblib.dump(clf_svc_calibrated, 'CalibratedClassifierCV=10_2.joblib')

['CalibratedClassifierCV=10.joblib']

In [ ]:
train = np.mean(clf_svc_calibrated.predict(train["document"].apply(lambda x: np.str_(x))) == train["label"])
test = np.mean(clf_svc_calibrated.predict(test["document"].apply(lambda x: np.str_(x))) == test["label"])

In [ ]:
with open('CalibratedClassifierCV=10_2.txt', 'w') as f:
    f.write(str(train))
    f.write(str(test))
    f.close()

In [5]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
#CalibratedClassifierCV, cv=5
SVC_pipeline_Calibrated = Pipeline([
                ('tfidf',vectorize),
                ('clf', CalibratedClassifierCV(LinearSVC(), cv=5))
            ])
clf_svc_calibrated = SVC_pipeline_Calibrated.fit(train["document"].apply(lambda x: np.str_(x)), train["label"])

joblib.dump(clf_svc_calibrated, 'CalibratedClassifierCV=5_2.joblib')

['CalibratedClassifierCV=5.joblib']

In [ ]:
train = np.mean(clf_svc_calibrated.predict(train["document"].apply(lambda x: np.str_(x))) == train["label"])
test = np.mean(clf_svc_calibrated.predict(test["document"].apply(lambda x: np.str_(x))) == test["label"])

In [ ]:
with open('CalibratedClassifierCV=5_2.txt', 'w') as f:
    f.write(str(train))
    f.write(str(test))
    f.close()

In [5]:
#SVC(kernel ='Linear', gamma = ‘scale’)
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

SVC_pipeline_Linear = Pipeline([
                ('tfidf', vectorize),
                ('clf', OneVsRestClassifier(SVC(kernel='linear', probability=True, class_weight='balanced', gamma = 'scale'), n_jobs=1))
            ])
clf_svc_linear = SVC_pipeline_Linear.fit(train["document"].apply(lambda x: np.str_(x)), train["label"])

joblib.dump(clf_svc_linear, 'SVC_kernel_Linear_2.joblib')

['SVC_kernel_Linear.joblib']

In [ ]:
train = np.mean(clf_svc_linear.predict(train["document"].apply(lambda x: np.str_(x))) == train["label"])
test = np.mean(clf_svc_linear.predict(test["document"].apply(lambda x: np.str_(x))) == test["label"])

In [ ]:
with open('SVC_kernel_Linear_2.txt', 'w') as f:
    f.write(str(train))
    f.write(str(test))
    f.close()